In [1]:
import os
import pickle

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Flatten, Dense, Reshape, Conv2DTranspose, Activation, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import numpy as np
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Keras version:", tf.keras.__version__)

if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available.")

tf.compat.v1.disable_eager_execution()

2024-11-22 08:43:03.299895: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-22 08:43:03.299927: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.4.0
Keras version: 2.4.0
GPU is not available.


2024-11-22 08:43:04.286623: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-11-22 08:43:04.286768: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-11-22 08:43:04.286782: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-22 08:43:04.286803: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-c8a454): /proc/driver/nvidia/version does not exist


# Data Preprocessing

In [14]:
import librosa
import numpy as np
import os
from midi2audio import FluidSynth
import pickle

In [12]:
def midi_to_wav(midi_file, wav_file, soundfont_path):
    fs = FluidSynth(soundfont_path)  # Load the soundfont
    fs.midi_to_audio(midi_file, wav_file)  # Convert MIDI to WAV
    print(f"Converted {midi_file} to {wav_file}")

In [16]:
sound_file_counter = 0
soundfont_path = "FluidR3_GM.sf2"
wav_path = "maestro_dataset/mp3/"

for path, _ , file_names in os.walk('maestro_dataset/midi'):
    for file_name in file_names:
        midi_file = os.path.join(path, file_name)
        wav_file = os.path.join(wav_path, f"sound_file_{sound_file_counter}.wav")
        midi_to_wav(midi_file, wav_file, soundfont_path)

        sound_file_counter += 1

Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'maestro_dataset/mp3/sound_file_0.wav'..
Converted maestro_dataset/midi/MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_04_Track04_wav.midi to maestro_dataset/mp3/sound_file_0.wav


Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'maestro_dataset/mp3/sound_file_1.wav'..
Converted maestro_dataset/midi/MIDI-Unprocessed_XP_17_R2_2004_01_ORIG_MID--AUDIO_17_R2_2004_03_Track03_wav.midi to maestro_dataset/mp3/sound_file_1.wav


Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.


...

FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'maestro_dataset/mp3/sound_file_31.wav'..
Converted maestro_dataset/midi/MIDI-Unprocessed_SMF_22_R1_2004_01-04_ORIG_MID--AUDIO_22_R1_2004_03_Track03_wav.midi to maestro_dataset/mp3/sound_file_31.wav


Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'maestro_dataset/mp3/sound_file_32.wav'..
Converted maestro_dataset/midi/MIDI-Unprocessed_XP_08_R1_2004_04-06_ORIG_MID--AUDIO_08_R1_2004_05_Track05_wav--1.midi to maestro_dataset/mp3/sound_file_32.wav


Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.


KeyboardInterrupt: 

In [24]:
class Loader:
    """Loader is responsible for loading an audio file."""

    def __init__(self, sample_rate, duration, mono):
        self.sample_rate = sample_rate
        self.duration = duration
        self.mono = mono

    def load(self, file_path):
        signal = librosa.load(file_path,
                              sr=self.sample_rate,
                              duration=self.duration,
                              mono=self.mono)[0]
        return signal

class Padder:
    """Padder is responsible to apply padding to an array."""

    def __init__(self, mode="constant"):
        self.mode = mode

    def left_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (num_missing_items, 0),
                              mode=self.mode)
        return padded_array

    def right_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (0, num_missing_items),
                              mode=self.mode)
        return padded_array


class LogSpectrogramExtractor:
    """LogSpectrogramExtractor extracts log spectrograms (in dB) from a
    time-series signal.
    """

    def __init__(self, frame_size, hop_length):
        self.frame_size = frame_size
        self.hop_length = hop_length

    def extract(self, signal):
        stft = librosa.stft(signal,
                            n_fft=self.frame_size,
                            hop_length=self.hop_length)[:-1]
        spectrogram = np.abs(stft)
        log_spectrogram = librosa.amplitude_to_db(spectrogram)
        return log_spectrogram


class MinMaxNormaliser:
    """MinMaxNormaliser applies min max normalisation to an array."""

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def normalise(self, array):
        norm_array = (array - array.min()) / (array.max() - array.min())
        norm_array = norm_array * (self.max - self.min) + self.min
        return norm_array

    def denormalise(self, norm_array, original_min, original_max):
        array = (norm_array - self.min) / (self.max - self.min)
        array = array * (original_max - original_min) + original_min
        return array


class Saver:
    """saver is responsible to save features, and the min max values."""

    def __init__(self, feature_save_dir, min_max_values_save_dir):
        self.feature_save_dir = feature_save_dir
        self.min_max_values_save_dir = min_max_values_save_dir

    def save_feature(self, feature, file_path):
        save_path = self._generate_save_path(file_path)
        np.save(save_path, feature)

    def save_min_max_values(self, min_max_values):
        save_path = os.path.join(self.min_max_values_save_dir,
                                 "min_max_values.pkl")
        self._save(min_max_values, save_path)

    @staticmethod
    def _save(data, save_path):
        with open(save_path, "wb") as f:
            pickle.dump(data, f)

    def _generate_save_path(self, file_path):
        file_name = os.path.split(file_path)[1]
        save_path = os.path.join(self.feature_save_dir, file_name + ".npy")
        return save_path


class PreprocessingPipeline:
    """PreprocessingPipeline processes audio files in a directory, applying
    the following steps to each file:
        1- load a file
        2- pad the signal (if necessary)
        3- extracting log spectrogram from signal
        4- normalise spectrogram
        5- save the normalised spectrogram

    Storing the min max values for all the log spectrograms.
    """

    def __init__(self):
        self.padder = None
        self.extractor = None
        self.normaliser = None
        self.saver = None
        self.min_max_values = {}
        self._loader = None
        self._num_expected_samples = None

    @property
    def loader(self):
        return self._loader

    @loader.setter
    def loader(self, loader):
        self._loader = loader
        self._num_expected_samples = int(loader.sample_rate * loader.duration)

    def process(self, audio_files_dir):
        for root, _, files in os.walk(audio_files_dir):
            for file in files:
                file_path = os.path.join(root, file)
                self._process_file(file_path)
                print(f"Processed file {file_path}")
        self.saver.save_min_max_values(self.min_max_values)

    def _process_file(self, file_path):
        signal = self.loader.load(file_path)
        if self._is_padding_necessary(signal):
            signal = self._apply_padding(signal)
        feature = self.extractor.extract(signal)
        norm_feature = self.normaliser.normalise(feature)
        save_path = self.saver.save_feature(norm_feature, file_path)
        self._store_min_max_value(save_path, feature.min(), feature.max())

    def _is_padding_necessary(self, signal):
        if len(signal) < self._num_expected_samples:
            return True
        return False

    def _apply_padding(self, signal):
        num_missing_samples = self._num_expected_samples - len(signal)
        padded_signal = self.padder.right_pad(signal, num_missing_samples)
        return padded_signal

    def _store_min_max_value(self, save_path, min_val, max_val):
        self.min_max_values[save_path] = {
            "min": min_val,
            "max": max_val
        }

In [37]:
# FRAME_SIZE = 512
# HOP_LENGTH = 256
# DURATION = 30.0  # in seconds
# SAMPLE_RATE = 22050
# MONO = True

# SPECTROGRAMS_SAVE_DIR = "maestro_dataset/spectograms"
# MIN_MAX_VALUES_SAVE_DIR = "maestro_dataset/minmax"
# FILES_DIR = "maestro_dataset/mp3"

# # instantiate all objects
# loader = Loader(SAMPLE_RATE, DURATION, MONO)
# padder = Padder()
# log_spectrogram_extractor = LogSpectrogramExtractor(FRAME_SIZE, HOP_LENGTH)
# min_max_normaliser = MinMaxNormaliser(0, 1)
# saver = Saver(SPECTROGRAMS_SAVE_DIR, MIN_MAX_VALUES_SAVE_DIR)

# preprocessing_pipeline = PreprocessingPipeline()
# preprocessing_pipeline.loader = loader
# preprocessing_pipeline.padder = padder
# preprocessing_pipeline.extractor = log_spectrogram_extractor
# preprocessing_pipeline.normaliser = min_max_normaliser
# preprocessing_pipeline.saver = saver

# preprocessing_pipeline.process(FILES_DIR)

FRAME_SIZE = 512
HOP_LENGTH = 256
DURATION = 0.74  # in seconds
SAMPLE_RATE = 22050
MONO = True

SPECTROGRAMS_SAVE_DIR = "maestro_dataset/spectograms"
MIN_MAX_VALUES_SAVE_DIR = "maestro_dataset/minmax"
FILES_DIR = "maestro_dataset/mp3"

# instantiate all objects
loader = Loader(SAMPLE_RATE, DURATION, MONO)
padder = Padder()
log_spectrogram_extractor = LogSpectrogramExtractor(FRAME_SIZE, HOP_LENGTH)
min_max_normaliser = MinMaxNormaliser(0, 1)
saver = Saver(SPECTROGRAMS_SAVE_DIR, MIN_MAX_VALUES_SAVE_DIR)

preprocessing_pipeline = PreprocessingPipeline()
preprocessing_pipeline.loader = loader
preprocessing_pipeline.padder = padder
preprocessing_pipeline.extractor = log_spectrogram_extractor
preprocessing_pipeline.normaliser = min_max_normaliser
preprocessing_pipeline.saver = saver

preprocessing_pipeline.process(FILES_DIR)

Processed file maestro_dataset/mp3/sound_file_21.wav
Processed file maestro_dataset/mp3/sound_file_14.wav
Processed file maestro_dataset/mp3/sound_file_27.wav
Processed file maestro_dataset/mp3/sound_file_15.wav
Processed file maestro_dataset/mp3/sound_file_17.wav
Processed file maestro_dataset/mp3/sound_file_8.wav
Processed file maestro_dataset/mp3/sound_file_6.wav
Processed file maestro_dataset/mp3/sound_file_19.wav
Processed file maestro_dataset/mp3/sound_file_20.wav
Processed file maestro_dataset/mp3/sound_file_13.wav
Processed file maestro_dataset/mp3/sound_file_23.wav
Processed file maestro_dataset/mp3/sound_file_3.wav
Processed file maestro_dataset/mp3/sound_file_33.wav
Processed file maestro_dataset/mp3/sound_file_12.wav
Processed file maestro_dataset/mp3/sound_file_5.wav
Processed file maestro_dataset/mp3/sound_file_0.wav
Processed file maestro_dataset/mp3/sound_file_10.wav
Processed file maestro_dataset/mp3/sound_file_31.wav
Processed file maestro_dataset/mp3/sound_file_16.wa

# Variational Autoencoder Model

In [28]:
def _calculate_reconstruction_loss(y_target, y_predicted):
    error = y_target - y_predicted
    reconstruction_loss = K.mean(K.square(error), axis=[1, 2, 3])
    return reconstruction_loss


def calculate_kl_loss(model):
    # wrap `_calculate_kl_loss` such that it takes the model as an argument,
    # returns a function which can take arbitrary number of arguments
    # (for compatibility with `metrics` and utility in the loss function)
    # and returns the kl loss
    def _calculate_kl_loss(*args):
        kl_loss = -0.5 * K.sum(1 + model.log_variance - K.square(model.mu) -
                               K.exp(model.log_variance), axis=1)
        return kl_loss
    return _calculate_kl_loss


class VAE:
    """
    VAE represents a Deep Convolutional variational autoencoder architecture
    with mirrored encoder and decoder components.
    """

    def __init__(self,
                 input_shape,
                 conv_filters,
                 conv_kernels,
                 conv_strides,
                 latent_space_dim):
        self.input_shape = input_shape # [28, 28, 1]
        self.conv_filters = conv_filters # [2, 4, 8]
        self.conv_kernels = conv_kernels # [3, 5, 3]
        self.conv_strides = conv_strides # [1, 2, 2]
        self.latent_space_dim = latent_space_dim # 2
        self.reconstruction_loss_weight = 1000

        self.encoder = None
        self.decoder = None
        self.model = None

        self._num_conv_layers = len(conv_filters)
        self._shape_before_bottleneck = None
        self._model_input = None

        self._build()

    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()

    def compile(self, learning_rate=0.0001):
        optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss=self._calculate_combined_loss,
                           metrics=[_calculate_reconstruction_loss,
                                    calculate_kl_loss(self)])

    def train(self, x_train, batch_size, num_epochs):
        self.model.fit(x_train,
                       x_train,
                       batch_size=batch_size,
                       epochs=num_epochs,
                       shuffle=True)

    def save(self, save_folder="."):
        self._create_folder_if_it_doesnt_exist(save_folder)
        self._save_parameters(save_folder)
        self._save_weights(save_folder)

    def load_weights(self, weights_path):
        self.model.load_weights(weights_path)

    def reconstruct(self, images):
        latent_representations = self.encoder.predict(images)
        reconstructed_images = self.decoder.predict(latent_representations)
        return reconstructed_images, latent_representations

    @classmethod
    def load(cls, save_folder="."):
        parameters_path = os.path.join(save_folder, "parameters.pkl")
        with open(parameters_path, "rb") as f:
            parameters = pickle.load(f)
        autoencoder = VAE(*parameters)
        weights_path = os.path.join(save_folder, "weights.h5")
        autoencoder.load_weights(weights_path)
        return autoencoder

    def _calculate_combined_loss(self, y_target, y_predicted):
        reconstruction_loss = self._calculate_reconstruction_loss(y_target, y_predicted)
        kl_loss = self._calculate_kl_loss(y_target, y_predicted)
        combined_loss = self.reconstruction_loss_weight * reconstruction_loss\
                                                         + kl_loss
        return combined_loss

    def _calculate_reconstruction_loss(self, y_target, y_predicted):
        error = y_target - y_predicted
        reconstruction_loss = K.mean(K.square(error), axis=[1, 2, 3])
        return reconstruction_loss

    def _calculate_kl_loss(self, y_target, y_predicted):
        kl_loss = -0.5 * K.sum(1 + self.log_variance - K.square(self.mu) -
                               K.exp(self.log_variance), axis=1)
        return kl_loss

    def _create_folder_if_it_doesnt_exist(self, folder):
        if not os.path.exists(folder):
            os.makedirs(folder)

    def _save_parameters(self, save_folder):
        parameters = [
            self.input_shape,
            self.conv_filters,
            self.conv_kernels,
            self.conv_strides,
            self.latent_space_dim
        ]
        save_path = os.path.join(save_folder, "parameters.pkl")
        with open(save_path, "wb") as f:
            pickle.dump(parameters, f)

    def _save_weights(self, save_folder):
        save_path = os.path.join(save_folder, "weights.h5")
        self.model.save_weights(save_path)

    def _build(self):
        self._build_encoder()
        self._build_decoder()
        self._build_autoencoder()

    def _build_autoencoder(self):
        model_input = self._model_input
        model_output = self.decoder(self.encoder(model_input))
        self.model = Model(model_input, model_output, name="autoencoder")

    def _build_decoder(self):
        decoder_input = self._add_decoder_input()
        dense_layer = self._add_dense_layer(decoder_input)
        reshape_layer = self._add_reshape_layer(dense_layer)
        conv_transpose_layers = self._add_conv_transpose_layers(reshape_layer)
        decoder_output = self._add_decoder_output(conv_transpose_layers)
        self.decoder = Model(decoder_input, decoder_output, name="decoder")

    def _add_decoder_input(self):
        return Input(shape=self.latent_space_dim, name="decoder_input")

    def _add_dense_layer(self, decoder_input):
        num_neurons = np.prod(self._shape_before_bottleneck) # [1, 2, 4] -> 8
        dense_layer = Dense(num_neurons, name="decoder_dense")(decoder_input)
        return dense_layer

    def _add_reshape_layer(self, dense_layer):
        return Reshape(self._shape_before_bottleneck)(dense_layer)

    def _add_conv_transpose_layers(self, x):
        """Add conv transpose blocks."""
        # loop through all the conv layers in reverse order and stop at the
        # first layer
        for layer_index in reversed(range(1, self._num_conv_layers)):
            x = self._add_conv_transpose_layer(layer_index, x)
        return x

    def _add_conv_transpose_layer(self, layer_index, x):
        layer_num = self._num_conv_layers - layer_index
        conv_transpose_layer = Conv2DTranspose(
            filters=self.conv_filters[layer_index],
            kernel_size=self.conv_kernels[layer_index],
            strides=self.conv_strides[layer_index],
            padding="same",
            name=f"decoder_conv_transpose_layer_{layer_num}"
        )
        x = conv_transpose_layer(x)
        x = ReLU(name=f"decoder_relu_{layer_num}")(x)
        x = BatchNormalization(name=f"decoder_bn_{layer_num}")(x)
        return x

    def _add_decoder_output(self, x):
        conv_transpose_layer = Conv2DTranspose(
            filters=1,
            kernel_size=self.conv_kernels[0],
            strides=self.conv_strides[0],
            padding="same",
            name=f"decoder_conv_transpose_layer_{self._num_conv_layers}"
        )
        x = conv_transpose_layer(x)
        output_layer = Activation("sigmoid", name="sigmoid_layer")(x)
        return output_layer

    def _build_encoder(self):
        encoder_input = self._add_encoder_input()
        conv_layers = self._add_conv_layers(encoder_input)
        bottleneck = self._add_bottleneck(conv_layers)
        self._model_input = encoder_input
        self.encoder = Model(encoder_input, bottleneck, name="encoder")

    def _add_encoder_input(self):
        return Input(shape=self.input_shape, name="encoder_input")

    def _add_conv_layers(self, encoder_input):
        """Create all convolutional blocks in encoder."""
        x = encoder_input
        for layer_index in range(self._num_conv_layers):
            x = self._add_conv_layer(layer_index, x)
        return x

    def _add_conv_layer(self, layer_index, x):
        """Add a convolutional block to a graph of layers, consisting of
        conv 2d + ReLU + batch normalization.
        """
        layer_number = layer_index + 1
        conv_layer = Conv2D(
            filters=self.conv_filters[layer_index],
            kernel_size=self.conv_kernels[layer_index],
            strides=self.conv_strides[layer_index],
            padding="same",
            name=f"encoder_conv_layer_{layer_number}"
        )
        x = conv_layer(x)
        x = ReLU(name=f"encoder_relu_{layer_number}")(x)
        x = BatchNormalization(name=f"encoder_bn_{layer_number}")(x)
        return x

    def _add_bottleneck(self, x):
        """Flatten data and add bottleneck with Guassian sampling (Dense
        layer).
        """
        self._shape_before_bottleneck = K.int_shape(x)[1:]
        x = Flatten()(x)
        self.mu = Dense(self.latent_space_dim, name="mu")(x)
        self.log_variance = Dense(self.latent_space_dim,
                                  name="log_variance")(x)

        def sample_point_from_normal_distribution(args):
            mu, log_variance = args
            epsilon = K.random_normal(shape=K.shape(self.mu), mean=0.,
                                      stddev=1.)
            sampled_point = mu + K.exp(log_variance / 2) * epsilon
            return sampled_point

        print(self.input_shape)

        x = Lambda(sample_point_from_normal_distribution,
               name="encoder_output")([self.mu, self.log_variance])
        return x


# Train

In [46]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
EPOCHS = 150

SPECTROGRAMS_PATH = "maestro_dataset/spectograms"


def load_fsdd(spectrograms_path):
    x_train = []
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames, 1)
            x_train.append(spectrogram)
    x_train = np.array(x_train)
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)
    print(x_train.shape)
    print(x_train)
    return x_train


def train(x_train, learning_rate, batch_size, epochs):
    autoencoder = VAE(
        input_shape=(256, 64, 1),
        conv_filters=(512, 256, 128, 64, 32),
        conv_kernels=(3, 3, 3, 3, 3),
        conv_strides=(2, 2, 2, 2, (2, 1)),
        latent_space_dim=128
    )
    autoencoder.summary()
    autoencoder.compile(learning_rate)
    autoencoder.train(x_train, batch_size, epochs)
    return autoencoder

In [47]:
x_train = load_fsdd(SPECTROGRAMS_PATH)
autoencoder = train(x_train, LEARNING_RATE, BATCH_SIZE, EPOCHS)
autoencoder.save("model")

(34, 256, 64, 1)
[[[[0.814349  ]
   [0.9856756 ]
   [1.        ]
   ...
   [1.        ]
   [1.        ]
   [0.9978009 ]]

  [[0.7738697 ]
   [0.8994985 ]
   [0.8959378 ]
   ...
   [0.8959378 ]
   [0.8959378 ]
   [0.89942753]]

  [[0.65179914]
   [0.588238  ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.3570581 ]]

  ...

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]]


 [[[0.814349  ]
   [0.9856756 ]
   [1.        ]
   ...
   [1.        ]
   [1.        ]
   [0.9978009 ]]

  [[0.7738697 ]
   [0.8994985 ]
   [0.8959378 ]
   ...
   [0.8959378 ]
   [0.8959378 ]
   [0.89942753]]

  [[0.65179914]
   [0.588238  ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.3570581 ]]

  ...

  [[0.        ]
   [0.

Train on 34 samples


2024-11-22 09:31:42.939869: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-11-22 09:31:43.139259: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2024-11-22 09:31:43.398416: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2445430000 Hz
2024-11-22 09:31:44.302995: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 21233664 exceeds 10% of free system memory.
2024-11-22 09:31:44.338601: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 21233664 exceeds 10% of free system memory.
2024-11-22 09:31:44.360760: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 21233664 exceeds 10% of free system memory.


Epoch 1/150


2024-11-22 09:31:45.512353: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 285212672 exceeds 10% of free system memory.
2024-11-22 09:31:45.512353: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 285212672 exceeds 10% of free system memory.


34/34 [==============================] - 6s 175ms/sample - loss: 1070.9966 - _calculate_reconstruction_loss: 0.2364 - _calculate_kl_loss: 834.5837
Epoch 2/150
34/34 [==============================] - 5s 140ms/sample - loss: 503.6679 - _calculate_reconstruction_loss: 0.2342 - _calculate_kl_loss: 269.4602
Epoch 3/150
34/34 [==============================] - 5s 142ms/sample - loss: 376.3226 - _calculate_reconstruction_loss: 0.2377 - _calculate_kl_loss: 138.6281
Epoch 4/150
34/34 [==============================] - 5s 138ms/sample - loss: 328.1326 - _calculate_reconstruction_loss: 0.2266 - _calculate_kl_loss: 101.5229
Epoch 5/150
34/34 [==============================] - 5s 139ms/sample - loss: 296.2864 - _calculate_reconstruction_loss: 0.2170 - _calculate_kl_loss: 79.3179
Epoch 6/150
34/34 [==============================] - 5s 137ms/sample - loss: 267.9529 - _calculate_reconstruction_loss: 0.2038 - _calculate_kl_loss: 64.1779
Epoch 7/150
34/34 [==============================] - 5s 138ms/sam

34/34 [==============================] - 4s 132ms/sample - loss: 62.8985 - _calculate_reconstruction_loss: 0.0573 - _calculate_kl_loss: 5.5574
Epoch 54/150
34/34 [==============================] - 5s 134ms/sample - loss: 62.4132 - _calculate_reconstruction_loss: 0.0559 - _calculate_kl_loss: 6.4702
Epoch 55/150
34/34 [==============================] - 4s 132ms/sample - loss: 61.4552 - _calculate_reconstruction_loss: 0.0544 - _calculate_kl_loss: 7.0607
Epoch 56/150
34/34 [==============================] - 4s 131ms/sample - loss: 61.4977 - _calculate_reconstruction_loss: 0.0546 - _calculate_kl_loss: 6.9232
Epoch 57/150
34/34 [==============================] - 5s 133ms/sample - loss: 59.9873 - _calculate_reconstruction_loss: 0.0533 - _calculate_kl_loss: 6.7222
Epoch 58/150
34/34 [==============================] - 4s 132ms/sample - loss: 58.7731 - _calculate_reconstruction_loss: 0.0526 - _calculate_kl_loss: 6.2016
Epoch 59/150
34/34 [==============================] - 5s 133ms/sample - loss:

34/34 [==============================] - 4s 132ms/sample - loss: 49.6621 - _calculate_reconstruction_loss: 0.0370 - _calculate_kl_loss: 12.6873
Epoch 106/150
34/34 [==============================] - 5s 134ms/sample - loss: 52.7075 - _calculate_reconstruction_loss: 0.0369 - _calculate_kl_loss: 15.7842
Epoch 107/150
34/34 [==============================] - 4s 131ms/sample - loss: 53.3133 - _calculate_reconstruction_loss: 0.0369 - _calculate_kl_loss: 16.3650
Epoch 108/150
34/34 [==============================] - 5s 133ms/sample - loss: 51.0456 - _calculate_reconstruction_loss: 0.0363 - _calculate_kl_loss: 14.7495
Epoch 109/150
34/34 [==============================] - 4s 132ms/sample - loss: 48.9320 - _calculate_reconstruction_loss: 0.0369 - _calculate_kl_loss: 12.0603
Epoch 110/150
34/34 [==============================] - 5s 133ms/sample - loss: 45.4926 - _calculate_reconstruction_loss: 0.0361 - _calculate_kl_loss: 9.4288
Epoch 111/150
34/34 [==============================] - 4s 132ms/sam